In [78]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint
import pandas as pd
import cv2
import numpy as np
import glob
import os

# Changing yellow background to white

In [76]:
directorio_actas_originales = glob.glob("/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/*.jpg")
for acta in directorio_actas_originales:
    filename = os.path.basename(acta)
    img = cv2.imread(acta)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    mask = cv2.bitwise_not(mask)
    cv2.imwrite('/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central_black_white/'+filename, mask)

['/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_21_mesa_254.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_54_mesa_713.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_19_mesa_239.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_9_mesa_123.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_49_mesa_626.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_49_mesa_632.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_58_mesa_767.jpg',
 '/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_53_mesa_707.jpg',
 '/Users/

# OCR processing

In [71]:


credenciales = pd.read_csv("/Users/rudikrompich/dialogos_storage/Rudik_extracting_accessKeys.csv")

def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}

                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '
    return text


def get_table_csv_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)



    # process using image bytes
    # get the results
    client = boto3.client('textract',
                      region_name='us-east-1',
                      aws_access_key_id=credenciales.iloc[0,0],
                      aws_secret_access_key=credenciales.iloc[0,1])



    response = client.analyze_document(Document={'Bytes':  bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks=response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)

    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():

        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'

    csv += '\n\n\n'
    return csv

def main(file_name):
    table_csv = get_table_csv_results(file_name)

    output_file = 'output666.csv'

    # replace content
    with open(output_file, "wt") as fout:
        fout.write(table_csv)

    # show the results
    #print('CSV OUTPUT FILE: ', output_file)



In [73]:
main("/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central_black_white/acta_centro_8_mesa_103.jpg")

Image loaded /Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central_black_white/acta_centro_8_mesa_103.jpg
[{'BlockType': 'PAGE',
  'Geometry': {'BoundingBox': {'Height': 1.0,
                               'Left': 0.0,
                               'Top': 0.0,
                               'Width': 1.0},
               'Polygon': [{'X': 0.0, 'Y': 0.0024619335308670998},
                           {'X': 0.9939146041870117, 'Y': 0.0},
                           {'X': 1.0, 'Y': 0.9994729161262512},
                           {'X': 0.0032511455938220024, 'Y': 1.0}]},
  'Id': 'f9ee19f9-d3f4-4933-9737-408297ceba0b',
  'Relationships': [{'Ids': ['05a8d8b8-3a4d-40a6-b004-09c2da7d9746',
                             'dd7c0674-2234-4cbb-9f08-2d07a5e212b0',
                             'dfbd4357-e8ee-4b35-8db5-cf55c6c73e85',
                             '6b453776-3e3c-47e3-a0d4-058a94b15544',
                             '1bd2c634-8092-41cf-9cd0-9f3e6398ae58